The goal of this notebook is to try AI Search in Azure.

See: https://python.langchain.com/v0.2/docs/integrations/vectorstores/azuresearch/

In [1]:
import os
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores.azuresearch import AzureSearch
from dotenv import load_dotenv, find_dotenv

In [2]:
_ = load_dotenv(find_dotenv(filename='.env'))

In [3]:
openai_api_key = os.getenv("OPENAI_API_KEY")
openai_api_version = "2023-05-15"
model = "text-embedding-ada-002"

In [5]:
vector_store_address = "https://ai-search-1-sn.search.windows.net"
vector_store_password = os.getenv("AZURE_SEARCH_ADMIN_KEY")

In [6]:
embeddings = OpenAIEmbeddings(
    openai_api_key=openai_api_key, 
    openai_api_version=openai_api_version, 
    model=model
)

In [7]:
index_name = "vector-1723032578840"
vector_store = AzureSearch(
    azure_search_endpoint=vector_store_address,
    azure_search_key=vector_store_password,
    index_name=index_name,
    embedding_function=embeddings.embed_query,
)

In [8]:
docs = vector_store.similarity_search(
    query="What is Tavily?",
    k=3,
    search_type="similarity",
)
print(docs[0].page_content)

HttpResponseError: (InvalidRequestParameter) Unknown field 'content_vector' in vector field list.
Code: InvalidRequestParameter
Message: Unknown field 'content_vector' in vector field list.
Exception Details:	(UnknownField) Unknown field 'content_vector' in vector field list.
	Code: UnknownField
	Message: Unknown field 'content_vector' in vector field list.